In [1]:
import pandas as pd
import numpy as np

DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data'

# N_FOLD = 5
# N_FOLD = 4
N_FOLD = 6

EPOCH = 3
# EPOCH = 6
# EPOCH = 10

# MODEL = 'bert-base-uncased'
# MODEL = 'allenai/scibert_scivocab_uncased'
# MODEL = 'emilyalsentzer/Bio_ClinicalBERT'
# MODEL = 'bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12'
# MODEL = 'GanjinZero/UMLSBert_ENG'
# MODEL = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
# MODEL = 'lordtt13/COVID-SciBERT'
# MODEL = 'allenai/scibert_scivocab_cased'
MODEL = 'cambridgeltl/BioRedditBERT-uncased'
# MODEL = 'emilyalsentzer/Bio_Discharge_Summary_BERT'
# MODEL = 'bionlp/bluebert_pubmed_uncased_L-24_H-1024_A-16'
# MODEL = 'bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12'
# MODEL = 'jambo/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext-finetuned-renet' # error


# 医学論文の自動仕分けチャレンジ ベースライン

    Pytorch と BERT を使ったベースラインです。
    judgement の値をもとにした層別交差検証 (Stratified KFold) をします。
    title のみで学習・推論します。
    モデルの出力は 0 ~ 1 の実数を想定した 回帰予測 とします。
    モデルの出力をある境界値をもとに 0 と 1 にします。
        境界値は 学習データの judgement の 0 と 1 の割合で決めます。
    交差検証で生成した5つのモデルでそれぞれ推論を行い、結果を平均して、テストコードの推論結果とします。

## 前提

    Google Colab で動かすことを想定します。

## 事前準備

    Google Drive の マイドライブ/Datasets/signate-471 というフォルダに 本コンペのデータを配置してください。

## アウトプット

    oof_df.csv: 交差検証でのモデルの出力などが入っています。
    submission.csv: 提出用のファイルです。

## スコア

    CV (交差検証): 0.82746
    LB (リーダーボード): 0.8368283611616681

## 謝辞

    この notebook は Kaggle で [@yasufuminakama](https://www.kaggle.com/yasufuminakama) さんが別コンペで公開されていた notebook をベースにしています。


## データをドライブからコピー

In [2]:
import os
import sys

# from google.colab import drive
# drive.mount('/gdrive')

# !cp /gdrive/MyDrive/Datasets/signate-471/train.csv .
# !cp /gdrive/MyDrive/Datasets/signate-471/test.csv .
# !cp /gdrive/MyDrive/Datasets/signate-471/sample_submit.csv .

!cp "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/train.csv" .
!cp "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/test.csv" .
!cp "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/sample_submit.csv" .

## ライブラリ インストール、インポート

In [3]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 4.3 MB/s 
     |████████████████████████████████| 636 kB 62.2 MB/s 
     |████████████████████████████████| 895 kB 71.4 MB/s 
     |████████████████████████████████| 56 kB 6.0 MB/s 
     |████████████████████████████████| 3.3 MB 72.4 MB/s 


In [4]:
import math
import random
import time
import warnings

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as T
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

In [5]:
DATA_DIR = './'
OUTPUT_DIR = './'

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()

In [7]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed = 471
seed_torch(seed)

## データ読み込み

In [8]:
train = pd.read_csv(DATA_DIR + "train.csv")
test = pd.read_csv(DATA_DIR + "test.csv")
sub = pd.read_csv(DATA_DIR + "sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]

In [9]:
train.title = train.title + ' ' + train.abstract.fillna('')
test.title = test.title + ' ' + test.abstract.fillna('')

In [10]:
# この値を境に、モデルの出力を 0 と 1 にします。
border = len(train[train["judgement"] == 1]) / len(train["judgement"])
print(border)

0.023282372444280715


## 前処理

In [11]:
def get_train_data(train):

    # 交差検証 用の番号を振ります。
    Fold = StratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=seed)
    for n, (train_index, val_index) in enumerate(Fold.split(train, train["judgement"])):
        train.loc[val_index, "fold"] = int(n)
    train["fold"] = train["fold"].astype(np.uint8)

    return train

In [12]:
def get_test_data(test):
    return test

In [13]:
train = get_train_data(train)

## データセット 定義

In [14]:
class BaseDataset(Dataset):
    def __init__(self, df, model_name, include_labels=True):
        tokenizer = T.BertTokenizer.from_pretrained(model_name)

        self.df = df
        self.include_labels = include_labels

        self.title = df["title"].tolist()
        self.encoded = tokenizer.batch_encode_plus(
            self.title,
            padding = 'max_length',            
            # max_length = 72,
            max_length = 500,
            truncation = True,
            return_attention_mask=True
        )
        
        if self.include_labels:
            self.labels = df["judgement"].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.encoded['input_ids'][idx])
        attention_mask = torch.tensor(self.encoded['attention_mask'][idx])

        if self.include_labels:
            label = torch.tensor(self.labels[idx]).float()
            return input_ids, attention_mask, label

        return input_ids, attention_mask

## モデル 定義

In [15]:
class BaseModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()

        self.model = T.BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        out = self.model(input_ids=input_ids, attention_mask=attention_mask)
        out = self.sigmoid(out.logits).squeeze()

        return out

## ツール

In [16]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

## 学習 補助関数

In [17]:
def train_fn(train_loader, model, criterion, optimizer, epoch, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to train mode
    model.train()

    for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        y_preds = model(input_ids, attention_mask)

        loss = criterion(y_preds, labels)

        # record loss
        losses.update(loss.item(), batch_size)
        loss.backward()

        optimizer.step()

        if step % 100 == 0 or step == (len(train_loader) - 1):
            print(
                f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )

    return losses.avg

## 評価 補助関数

In [18]:
def valid_fn(valid_loader, model, criterion, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds = []

    for step, (input_ids, attention_mask, labels) in enumerate(valid_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        # compute loss
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask)

        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)

        # record score
        preds.append(y_preds.to("cpu").numpy())

        if step % 100 == 0 or step == (len(valid_loader) - 1):
            print(
                f"EVAL: [{step}/{len(valid_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )

    predictions = np.concatenate(preds)
    return losses.avg, predictions

## 推論関数

In [19]:
def inference():
    predictions = []

    test_dataset = BaseDataset(test, MODEL, include_labels=False)
    test_loader = DataLoader(
        test_dataset, batch_size=16, shuffle=False, num_workers=N_FOLD - 1, pin_memory=True
    )

    for fold in range(N_FOLD):
        LOGGER.info(f"========== model: fold: {fold} inference ==========")
        model = BaseModel(MODEL)
        model.to(device)
        model.load_state_dict(torch.load(OUTPUT_DIR + f"fold{fold}_best.pth")["model"])
        model.eval()
        preds = []
        for i, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            with torch.no_grad():
                y_preds = model(input_ids, attention_mask)
            preds.append(y_preds.to("cpu").numpy())
        preds = np.concatenate(preds)
        predictions.append(preds)
    predictions = np.mean(predictions, axis=0)

    return predictions

## 学習

In [20]:
def train_loop(train, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # Data Loader
    # ====================================================
    trn_idx = train[train["fold"] != fold].index
    val_idx = train[train["fold"] == fold].index

    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)

    train_dataset = BaseDataset(train_folds, MODEL)
    valid_dataset = BaseDataset(valid_folds, MODEL)

    train_loader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        # num_workers=4,
        num_workers=N_FOLD - 1,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=16,
        shuffle=False,
        # num_workers=4,
        num_workers=N_FOLD - 1,
        pin_memory=True,
        drop_last=False,
    )

    # ====================================================
    # Model
    # ====================================================
    model = BaseModel(MODEL)
    model.to(device)

    optimizer = T.AdamW(model.parameters(), lr=2e-5)

    criterion = nn.BCELoss()

    # ====================================================
    # Loop
    # ====================================================
    best_score = -1
    best_loss = np.inf

    for epoch in range(EPOCH):
        start_time = time.time()
        
        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        valid_labels = valid_folds["judgement"].values

        # scoring
        score = fbeta_score(valid_labels, np.where(preds < border, 0, 1), beta=7.0)

        elapsed = time.time() - start_time
        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
        )
        LOGGER.info(f"Epoch {epoch+1} - Score: {score}")

        if score > best_score:
            best_score = score
            LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
            torch.save(
                {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"fold{fold}_best.pth"
            )

    check_point = torch.load(OUTPUT_DIR + f"fold{fold}_best.pth")

    valid_folds["preds"] = check_point["preds"]

    return valid_folds

## メイン

In [21]:
def get_result(result_df):
    preds = result_df["preds"].values
    labels = result_df["judgement"].values
    score = fbeta_score(labels, np.where(preds < border, 0, 1), beta=7.0)
    LOGGER.info(f"Score: {score:<.5f}")

In [22]:
def main():
    # Training
    oof_df = pd.DataFrame()
    for fold in range(N_FOLD):
        _oof_df = train_loop(train, fold)
        oof_df = pd.concat([oof_df, _oof_df])
        LOGGER.info(f"========== fold: {fold} result ==========")
        get_result(_oof_df)
        
    # CV result
    LOGGER.info(f"========== CV ==========")
    get_result(oof_df)
    
    # Save OOF result
    oof_df.to_csv(OUTPUT_DIR + "oof_df.csv", index=False)

    # Inference
    predictions = inference()

    # submission_prob
    sub["judgement"] = predictions
    sub.to_csv(OUTPUT_DIR + "sub_prob_021.csv", index=False, header=False)

    predictions = np.where(predictions < border, 0, 1)

    # submission
    sub["judgement"] = predictions
    sub.to_csv(OUTPUT_DIR + "sub_021.csv", index=False, header=False)

In [23]:
if __name__ == "__main__":
    main()

========== fold: 0 training ==========


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

...

  0%|          | 0/2553 [00:00<?, ?it/s]

========== model: fold: 4 inference ==========
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cambridgeltl/BioRedditBERT-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2553 [00:00<?, ?it/s]

========== model: fold: 5 inference ==========
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cambridgeltl/BioRedditBERT-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2553 [00:00<?, ?it/s]

In [24]:
!cp ./sub_prob_021.csv "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01/sub_prob_021.csv"
!cp ./sub_021.csv "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01/sub_021.csv"

!cp ./fold0_best.pth "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01/021_fold0_best.pth"
!cp ./fold1_best.pth "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01/021_fold1_best.pth"
!cp ./fold2_best.pth "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01/021_fold2_best.pth"
!cp ./fold3_best.pth "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01/021_fold3_best.pth"
!cp ./fold4_best.pth "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01/021_fold4_best.pth"
!cp ./fold5_best.pth "/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01/021_fold5_best.pth"

医学論文の自動仕分けチャレンジ ベースライン

PyTorch と BERT を使ったベースラインです。

judgement の値をもとにした層別交差検証 (Stratified KFold) をします。

title のみで学習・推論します。

モデルの出力は 0 ~ 1 の実数を想定した 回帰予測 とします。

モデルの出力をある境界値をもとに 0 と 1 にします。

境界値は 学習データの judgement の 0 と 1 の割合で決めます。

交差検証で生成した5つのモデルでそれぞれ推論を行い、結果を平均して、テストコードの推論結果とします。